# Backtesting ML Regression-Based

## Load the model

In [25]:
import pickle

In [26]:
with open('models/model_dt_microsoft_linkedin_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [27]:
model_dt

DecisionTreeRegressor(max_depth=15)

In [ ]:
model_dt.predict(X=?)

## Load the data

In [30]:
import pandas as pd

df = pd.read_excel('data/Stock_Microsfot_x_Linkedin_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,diff_daily_1,diff_daily_1_direction,buy,buy_sell
Date,,,,,,,,,
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.882095,UP,1,1
2016-12-12,56.803028,57.244073,56.711145,57.124622,20198100,0.183765,UP,1,0
...,...,...,...,...,...,...,...,...,...
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,4.649994,UP,1,0
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800,10.760010,UP,1,0


## Backtesting.py Library

### Create your Strategy Class

In [12]:
from backtesting import Backtest, Strategy

In [13]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [16]:
df_explanatory.iloc[-1:, :]

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800


In [18]:
df.diff_daily_1.iloc[-1:]

Date
2023-03-16    10.76001
Name: diff_daily_1, dtype: float64

In [21]:
class SimpleClassificationUD(Strategy):
    def init(self):
        self.model = model_dt
        self.daily_direction = df.diff_daily_1
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[-1:, :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 5 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [ ]:
bt = Backtest(df_explanatory, SimpleClassificationUD,
              cash=10000, commission=.002, exclusive_orders=True)

results = bt.run()

### Interpret backtesting results

In [22]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-09 00:00:00
End,2023-03-16 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],85.850254
Equity Final [$],12766.063002
Equity Peak [$],26183.02207
Return [%],27.66063


### Visualize the strategy performance

In [24]:
bt.plot(filename='reports_backtesting/backtesting_regression_5.html')

Row(id='2428', ...)